## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-26-04-00-00 +0000,wsj,A Minnesota startup alleges a Chinese company—...,https://www.wsj.com/politics/national-security...
1,2025-11-26-03-59-51 +0000,bbc,Ukraine says 'understanding' reached on US pea...,https://www.bbc.com/news/articles/cy95jvw57v2o...
2,2025-11-26-03-54-29 +0000,nypost,Mike Johnson says leading the House is like be...,https://nypost.com/2025/11/25/us-news/mike-joh...
3,2025-11-26-03-32-08 +0000,nypost,"60-year-old skydiver plummets to ground, dies ...",https://nypost.com/2025/11/25/us-news/60-year-...
4,2025-11-26-03-31-59 +0000,bbc,'Once in 300 years' rain hits Thai city as flo...,https://www.bbc.com/news/articles/ckg97wx144jo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,45
15,ukraine,21
173,new,18
18,peace,17
36,year,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...,122
273,2025-11-25-12-56-44 +0000,nypost,"Trump Army secretary meets Russians after US, ...",https://nypost.com/2025/11/25/us-news/trump-ar...,120
167,2025-11-25-20-03-41 +0000,nypost,Trump dispatches top envoys to Russia and Ukra...,https://nypost.com/2025/11/25/us-news/trump-sa...,118
193,2025-11-25-18-50-55 +0000,nypost,How Army Secretary Dan Driscoll rose from Trum...,https://nypost.com/2025/11/25/us-news/how-army...,118
16,2025-11-26-02-22-37 +0000,wapo,"Trump pushes Ukraine, Russia on peace deal, bu...",https://www.washingtonpost.com/world/2025/11/2...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,122,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...
27,50,2025-11-26-01-25-44 +0000,bbc,New Zealand woman gets life sentence for 'suit...,https://www.bbc.com/news/articles/cp84xed0g18o...
229,47,2025-11-25-16-09-36 +0000,nypost,Ukraine peace plan expected to be rejected by ...,https://nypost.com/2025/11/25/world-news/ukrai...
142,47,2025-11-25-21-14-00 +0000,wsj,At the White House’s annual—and typically stai...,https://www.wsj.com/politics/policy/trump-pard...
13,45,2025-11-26-02-54-56 +0000,latimes,California cop allegedly stole cash from motor...,https://www.latimes.com/california/story/2025-...
241,44,2025-11-25-15-26-18 +0000,latimes,13-year-old girl arrested after fatally shooti...,https://www.latimes.com/california/story/2025-...
41,44,2025-11-26-00-42-29 +0000,nypost,Fashionable fliers take off with Sean Duffy’s ...,https://nypost.com/2025/11/25/us-news/fliers-p...
53,42,2025-11-26-00-09-20 +0000,nypost,Football coach Travis Turner is just latest st...,https://nypost.com/2025/11/25/us-news/coach-tr...
49,34,2025-11-26-00-22-48 +0000,nyt,Trump Administration Will Raise Prices for For...,https://www.nytimes.com/2025/11/25/us/politics...
129,32,2025-11-25-21-36-00 +0000,wsj,Autodesk CEO Sees Long-Term Growth Due to Stea...,https://www.wsj.com/business/earnings/autodesk...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
